## Import Library

In [29]:
import pandas as pd
import ta
import os
from dotenv import load_dotenv

from binance.um_futures import UMFutures

pd.options.mode.copy_on_write = True

## Obtain data

Read 2020 data and load into a dictionary of DataFrame

Use BTC as local currency

In [4]:
def get_symbols(filepath = '../Binance Data/symbols_list.txt'):
    """
    read .txt file to get the list of symbols
    returns a list of symbols
    """
    with open(filepath, 'r') as file:
        symbols = file.readlines()
    file.close()
    symbols = [symbol.strip() for symbol in symbols]
    return symbols

In [5]:
def get_coins(symbols, local_coin='BTC'):
    """
    returns a list of coins whose local coin is default to be BTC
    """
    coins = []
    for symbol in symbols:
        if symbol.endswith(local_coin):
            coins.append(symbol[:len(symbol)-len(local_coin)])
    return coins

In [6]:
def load_coins(coins: list,
              local_coin: str = 'BTC',
              interval: str = '15m',
              year: int = None):
    data_dict = {}
    for coin in coins:
        symbol = coin + local_coin
        if year is None:
            path = f'../Binance Data/{interval}/{symbol}.csv'
        else:
            path = f'../Binance Data/{interval}/{year}/{symbol}.csv'
        df = pd.read_csv(path)
        df.set_index('index', inplace=True)
        data_dict[symbol] = df
    return data_dict

In [10]:
local_coin = "USDT"
symbols = get_symbols()
coins = get_coins(symbols=symbols, local_coin=local_coin)
symbols = [coin + local_coin for coin in coins]
data = load_coins(coins=coins, local_coin=local_coin, year=2020)

## Implementation

### Step 1
Calculate RSI

In [11]:
def cal_rsi(df, period: int = 14):
    df['RSI'] = ta.momentum.RSIIndicator(df.Close,
                                         window=period).rsi()
    return df

In [12]:
def cal_return(df):
    df['Return'] = df['Close'].pct_change()
    return df

### Step 2
Calculate Benchmark

In [13]:
def cal_benchmark(data_dict):
    market_df = pd.concat([data_dict[symbol] for symbol in data_dict])
    market_df['Weighted Return'] = market_df['Return'] * market_df['Volume']
    weighted_return_total = market_df.groupby(market_df.index)['Weighted Return'].sum()
    volume_total = market_df.groupby(market_df.index)['Volume'].sum()
    benchmark = weighted_return_total / volume_total * 1000000
    # return data_dict[symbol]
    benchmark.index = data_dict[symbol].index
    benchmark.name = 'Benchmark Return'
    return benchmark

In [14]:
for symbol in data:
    data[symbol] = cal_rsi(data[symbol])
    data[symbol] = cal_return(data[symbol])

In [15]:
benchmark = cal_benchmark(data)

### Step 3
Calculate Beta

In [16]:
def cal_beta(data_dict: dict, benchmark, window: int = 30):
    for symbol in data_dict:
        # create a temp to avoid messing up the original data
        # use left join to join on date 
        temp = data_dict[symbol].merge(benchmark, on='index', how='left')
        covariance = temp['Return'].rolling(window).cov(temp['Benchmark Return'])
        variance = temp['Benchmark Return'].rolling(window).var()
        beta = covariance / variance
        data_dict[symbol]['beta'] = beta
    return data_dict

In [17]:
data_dict = cal_beta(data, benchmark)

### Step 4
Construct Beta-Neutral Portfolio

In [18]:
def create_streaming(data_dict, idx):
    data_stream = pd.DataFrame()
    for symbol in data_dict:
        data_stream[symbol] = data_dict[symbol].loc[idx]
    return data_stream.T
    

In [24]:
def construct_portfolio(df,
                                  rsi_long: int = 30,
                                  rsi_short: int = 70):
    # First check if can construct portfolio, or need to rebalance
    if df['RSI'].min() > rsi_long or df['RSI'].max() < rsi_short:
        return None
    
    # Establish long and short potision
    long_positions  = df[(df['RSI'] < rsi_long)]
    short_positions = df[(df['RSI'] > rsi_short)]

    long_positions['qty']  = rsi_long  - long_positions['RSI']
    short_positions['qty'] = rsi_short - short_positions['RSI']

    # Calculate and rebalance beta
    long_beta = sum(long_positions['qty'] * long_positions['beta'])
    short_beta = sum(short_positions['qty'] * short_positions['beta'])

    long_positions['qty'] *= - short_beta / long_beta

    qty = pd.concat([long_positions['qty'], short_positions['qty']])
    qty = qty / sum(qty)

    df = df.merge(qty, left_index=True, right_index=True, how='left')
    return df.fillna(0)

In [45]:
class Account():
    def __init__(self,
                 initial_balance: float = 1000000,
                 fee_rate: float = 0.00018, #0.018% for maker with BNB 10% off
                 ):
        self.balance = initial_balance
        self.fee_rate = fee_rate

        # fees are not included in the balance because BNB is used to pay the fee
        # BNB is not used as collateral

        self.fee = 0 

        self.collateral = 0
        self.positions = {}
    
    def get_balance(self):
        return self.balance
    
    def calculate_margin(self,
                         entry_price: float,
                         qty: float,
                         leverage: float):
        return (qty * entry_price) / leverage

    def open_position(self, 
                      symbol: str,
                      entry_price: float,
                      qty: float,
                      leverage: float,
                      is_long: bool):

        margin = self.calculate_margin(entry_price, qty, leverage)
        if margin > self.balance:
            raise ValueError("Not enough balance to open this position")
        
        self.balance -= margin
        self.collateral += margin

        position = Position(symbol,
                            ntry_price=entry_price,
                            qty=qty,
                            margin=margin,
                            is_long=is_long)
        self.positions[symbol] = position
    

    def close_position(self,
                       symbol: str,
                       close_price: float):
        if symbol not in self.positions or not self.positions[symbol].is_open:
            raise ValueError("No open position for this symbol")
        
        position = self.positions.pop(symbol)
        pnl = position.close(close_price)

        self.balance += position.margin
        self.collateral -= position.margin

        self.balance += pnl
        return pnl

    def get_inital_margin(self):

        return self.notional_position_value / self.leverage_level

    def get_maintenance_margin(self):

        # take 1% maintenance margin rate for prudence
        
        return self.notional_position_value * 0.01
    

In [44]:
class Position():
    def __init__(self,
                 symbol: str,
                 entry_price: float,
                 qty: float,
                 margin: float,
                 is_long: bool):
        self.symbol = symbol
        self.entry_price = entry_price
        self.qty = qty
        self.margin = margin
        self.is_long = is_long
        self.is_open = True
    
    def close(self,
              close_price):
        if self.is_long:
            pnl = (close_price - self.entry_price) * self.qty
        else:
            pnl = (self.entry_price - close_price) * self.qty
        
        self.is_open = False
        return pnl

    def __repr__(self):
        position = "long" if self.is_long else "short"
        return(f"Symbol = {self.symbol},  "
               f"entry_price = {self.entry_price}, "
               f"quantity = {self.quantity}, "
               f"margin = {self.margin}, "
               f"position = {position}, "
               f"is_open = {self.is_open}")
    

In [46]:
account = Account(initial_balance=1000)

In [ ]:
account.open_position(symbol="BTCUSDT",
                      entry_price=20000,
                      qty=0.1,
                      margin)

In [23]:
idxs = data['ETHBTC'].index

account_value = 1000000

for idx in idxs:
    data_stream = create_streaming(data, idx)

    # Check if there is any NaN in the indicators
    # Typically in the first few rows
    if data_stream.isna().any().any():
        continue

    portfolio = construct_portfolio(data_stream)
    # Check if can construct portfolio, or need to rebalance
    if portfolio is None:
        continue
    else:
        # Construct or Rebalance portfolio
        


SyntaxError: incomplete input (1392542997.py, line 19)

## Connect Binance API

In [34]:
def connectBinanceAPI():
    load_dotenv()
    API_KEY    = os.getenv('API_KEY')
    API_SECRET = os.getenv('API_SECRET')

    return UMFutures(API_KEY, API_SECRET)

In [35]:
um_futures_client = connectBinanceAPI()

## Defining Strategy

### Outline of the Strategy
Construct a market neutral strategy to long underpriced and short overpriced using RSI as indicator

## Parameters

## Backtesting

## Results

## Reference

https://chatgpt.com/c/67102c3c-47d4-800f-8932-a3001c34c2c8

#### Margin Requirement

https://www.binance.com/en/support/faq/how-to-calculate-the-margin-requirement-on-binance-futures-trading-4f83b65d83654b5e933f85ef12bf769d

# Test

In [36]:
import requests

In [40]:
# Define the base URL of the API
testnet_url = "https://testnet.binancefuture.com"
kline_request = "/fapi/v1/klines"

params = {
    "symbol": "BTCUSDT",
    "interval": "15m",
    "startTime": 
    "endTime":
}

# Send the GET request
response = requests.get(testnet_url + kline_request,
                        params = params)

# Check the response status and content
if response.status_code == 200:
    print("Success:", response.json())  # Assuming the response is in JSON format
else:
    print("Failed with status code:", response.status_code)

Success: [[1729415700000, '67391.70', '68066.00', '66550.30', '67661.70', '31.382', 1729416599999, '2112716.29830', 953, '13.956', '946417.81320', '0'], [1729416600000, '67526.70', '68066.00', '66550.60', '67661.70', '41.324', 1729417499999, '2788352.64440', 491, '12.410', '843603.08850', '0'], [1729417500000, '67661.70', '68066.00', '66550.40', '68066.00', '10.159', 1729418399999, '679908.03890', 353, '3.878', '260678.29210', '0'], [1729418400000, '67796.70', '68066.00', '66550.40', '68050.00', '26.471', 1729419299999, '1783715.76420', 585, '14.061', '955449.10100', '0'], [1729419300000, '68050.00', '68066.00', '66100.10', '67661.80', '48.494', 1729420199999, '3260675.63000', 1038, '24.670', '1673614.94720', '0'], [1729420200000, '67661.80', '68066.70', '66100.60', '66100.60', '38.487', 1729421099999, '2569309.51520', 714, '10.122', '684191.83910', '0'], [1729421100000, '66176.60', '67800.00', '66100.60', '66311.40', '34.826', 1729421999999, '2341249.14060', 752, '22.502', '1521019.24